In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import mpld3
#mpld3.enable_notebook()

In [ ]:
def normalise(means, errors, baseline, keep=False):
    new_means = {}
    new_errors = {}
    num_tests = len(means[baseline])
    for version in means:
        if version == baseline and not keep:
            continue
        new_means[version] = [means[version][i]/means[baseline][i] for i in range(num_tests)]
        new_errors[version] =[errors[version][i]/means[baseline][i] for i in range(num_tests)]
    return new_means, new_errors

In [ ]:
def bar_plot(ax, data, errors=None, colors=None, total_width=0.8, single_width=1, legend=False, capsize=3):
    """Draws a bar plot with multiple bars per data point.

    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.

    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.

        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }

    errors: dictionary, optional
        Dictionary of standard deviations, corresponding structure to data

    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)

    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.

    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.

    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """

    # Check if colors where provided, otherwhise use the default color cycle
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Number of bars per group
    n_bars = len(data)

    # The width of a single bar
    bar_width = total_width / n_bars

    # List containing handles for the drawn bars, used for the legend
    bars = []

    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2

        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            if errors is None:
                bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)])
            else:
                err = errors[name][x]
                bar = ax.bar(x + x_offset, y, yerr=err, error_kw=dict(capsize=capsize),
                             width=bar_width * single_width, color=colors[i % len(colors)])

        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])

#    # Draw legend if we need
#    if legend:
#        ax.legend(bars, data.keys())
    # return the handlers/labels for a legend
    if legend:
        return bars, data.keys()

## Specific pattern matching cases with significant improvement

In [ ]:
data = {}
# Check file to see formatting, has explanation between data
with open("../../wasm-of-ocaml/benchmarks/evaluation/patternMatching.txt") as f:
    for _ in range(2):
        f.readline()
    for _ in range(2):
        line = f.readline().strip().split()
        data[line[0]] = {"original" : 
          {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}}
    
    for _ in range(2):
        f.readline()
    for _ in range(2):
        line = f.readline().strip().split()
        data[line[0]]["optimised"] = \
          {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
    

In [ ]:
data

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,4))
ax = axs[0]
times = {version: [data[test][version]["time"] for test in ["splice", "splice2"]] for version in ["original", "optimised"]}
errors = {version: [data[test][version]["error"] for test in ["splice", "splice2"]] for version in ["original", "optimised"]}
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors, capsize=15)
ax.set_title("Execution time")
ax.set_xticks(range(2))
ax.set_xticklabels(["splice", "splice2"])
ax.set_ylabel("time (ms)");

ax = axs[1]
sizes = {version: [data[test][version]["filesize"] for test in ["splice", "splice2"]] for version in ["original", "optimised"]}
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks(range(2))
ax.set_xticklabels(["splice", "splice2"])
ax.set_ylabel("size (bytes)");

fig.legend(handlers, labels, loc='upper left');

## More general performance

In [ ]:
data = {}
# Check file to see formatting, has explanation between data
with open("../../wasm-of-ocaml/benchmarks/evaluation/patternMatching.txt") as f:
    line = ""
    while not line.startswith("-----"):
        line = f.readline()
    
    f.readline()
    line = f.readline().strip().split()
    while line != []:
        data[line[0]] = {"original" : 
          {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}}
        line = f.readline().strip().split()
    
    f.readline()
    line = f.readline().strip().split()
    while line != []:
        data[line[0]]["optimised"] = \
          {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
        line = f.readline().strip().split()

tests = list(data.keys())

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,4))
ax = axs[0]
times = {version: [data[test][version]["time"] for test in tests] for version in ["original", "optimised"]}
errors = {version: [data[test][version]["error"] for test in tests] for version in ["original", "optimised"]}
times, errors = normalise(times, errors, "original", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")
ax.set_ylim((0.7, 1.6))

ax = axs[1]
sizes = {version: [data[test][version]["filesize"] for test in tests] for version in ["original", "optimised"]}
handlers, labels = bar_plot(ax, sizes, legend=True)
ax.set_title("Filesize")
ax.set_xticks([])
ax.set_ylabel("size (bytes)");

fig.legend(handlers, labels, loc='upper left');
plt.subplots_adjust(left=0.15)

Negligible change in execution time. Filesize unchanged except for mergesort program, where it decreases very slightly. Overall, limited impact of changing to optimised version. Could possibly be more significant when very complex cases/datatypes involved, especially with guards.  
Filesize decrease is slightly more significant in the case of 'splice' programs, because that is a large part of the programs.